In [1]:
%run ../../src/start.py


python	3.9.6 (default, Jul 30 2021, 16:35:19) 
[GCC 7.5.0]
---------------------
Versions:
----------------------
pandas      1.3.1
numpy       1.20.3
matplotlib  3.4.2
seaborn     0.11.1
plotly      5.1.0
boto3       1.18.16
awswrangler 2.10.0
----------------------


Loaded Libraries
-------------------
import pandas as pd
import numpy as np
import sys,os
import re
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


## AWS
import boto3
import awswrangler as wr
----------------


GLOBAL VARIABLES
--------------------------
# Creating the low level functional client
client = boto3.client(
    's3',
    aws_access_key_id = AWS_KEY_ID,
    aws_secret_access_key = AWS_SECRET,
)
    
# Creating the high level object oriented interface
resource = boto3.resource(
    's3',
    aws_access_key_id = AWS_KEY_I

## Load Crime Data

In [2]:
# helper function
def bucket_raw_path(bucket_name,path_dir):
    ''''''
    raw_path = f's3://{bucket_name}/{path_dir}'
    return raw_path

In [3]:
raw = bucket_raw_path('dend-data',f'capstone/final-data/crime-data/')
raw

's3://dend-data/capstone/final-data/crime-data/'

## List Objects in bucket

In [4]:
wr.s3.list_objects(raw)

['s3://dend-data/capstone/final-data/crime-data/crime-09-18.csv']

# Create dataframe from S3

In [5]:
def s3_files_to_df(s3_files_path):
    ''' load s3 file path from wr result
    returns a dataframe of concat files '''
    df_list = []
    for i in wr.s3.list_objects(s3_files_path):
        temp = wr.s3.read_csv(i,)
        df_list.append(temp)
    # create df from list files
    df = pd.concat(df_list, ignore_index=True)
    return df

In [6]:
%%time
df_crime = s3_files_to_df(raw)

CPU times: user 3.36 s, sys: 868 ms, total: 4.23 s
Wall time: 59.4 s


In [7]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121231 entries, 0 to 1121230
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   date_time            1121231 non-null  object
 1   offenses             1121231 non-null  int64 
 2   offense_type         1121231 non-null  object
 3   block_range          1121231 non-null  object
 4   street_name          1121231 non-null  object
 5   type                 1121231 non-null  object
 6   suffix               1121231 non-null  object
 7   beat                 1121231 non-null  object
 8   premise              1121231 non-null  object
 9   date                 1121231 non-null  object
 10  hour                 1121231 non-null  int64 
 11  year                 1121231 non-null  int64 
 12  premise_description  1121231 non-null  object
dtypes: int64(3), object(10)
memory usage: 111.2+ MB


In [8]:
df_crime.loc[df_crime.duplicated(keep='first'),:]

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description


# Crime Data
Total rows: $1,121,230 $

In [9]:
df_crime.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description
0,2009-08-01 00:00:00,1,Robbery,3000-3099,durham,dr,N,3B30,210,08/01/09 00:00:00,0,2009,restaurant cafeteria
1,2009-08-01 01:00:00,1,Theft,6000-6099,richmond,ave,-,18F30,210,08/01/09 00:00:00,1,2009,restaurant cafeteria
2,2009-08-01 04:00:00,1,Burglary,3900-3999,broadway,st,-,13D20,210,08/01/09 00:00:00,4,2009,restaurant cafeteria
3,2009-08-01 05:00:00,1,Burglary,8500-8599,bellaire,blvd,-,18F60,210,08/01/09 00:00:00,5,2009,restaurant cafeteria
4,2009-08-01 19:00:00,1,Theft,1100-1199,uptown park,blvd,-,18F20,210,08/01/09 00:00:00,19,2009,restaurant cafeteria


In [10]:
df_crime.apply(lambda x: sum(x.isnull()))

date_time              0
offenses               0
offense_type           0
block_range            0
street_name            0
type                   0
suffix                 0
beat                   0
premise                0
date                   0
hour                   0
year                   0
premise_description    0
dtype: int64

In [11]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121231 entries, 0 to 1121230
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   date_time            1121231 non-null  object
 1   offenses             1121231 non-null  int64 
 2   offense_type         1121231 non-null  object
 3   block_range          1121231 non-null  object
 4   street_name          1121231 non-null  object
 5   type                 1121231 non-null  object
 6   suffix               1121231 non-null  object
 7   beat                 1121231 non-null  object
 8   premise              1121231 non-null  object
 9   date                 1121231 non-null  object
 10  hour                 1121231 non-null  int64 
 11  year                 1121231 non-null  int64 
 12  premise_description  1121231 non-null  object
dtypes: int64(3), object(10)
memory usage: 111.2+ MB


# set datetime as index

In [12]:
# set to datetime
df_crime.date_time = pd.to_datetime(df_crime.date_time)

In [ ]:
#df_crime = df_crime.set_index('date_time')

In [13]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121231 entries, 0 to 1121230
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   date_time            1121231 non-null  datetime64[ns]
 1   offenses             1121231 non-null  int64         
 2   offense_type         1121231 non-null  object        
 3   block_range          1121231 non-null  object        
 4   street_name          1121231 non-null  object        
 5   type                 1121231 non-null  object        
 6   suffix               1121231 non-null  object        
 7   beat                 1121231 non-null  object        
 8   premise              1121231 non-null  object        
 9   date                 1121231 non-null  object        
 10  hour                 1121231 non-null  int64         
 11  year                 1121231 non-null  int64         
 12  premise_description  1121231 non-null  object        
dt

In [50]:
df_crime.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description
0,2009-08-01 00:00:00,1,Robbery,3000-3099,durham,dr,N,3B30,210,08/01/09 00:00:00,0,2009,restaurant cafeteria
1,2009-08-01 01:00:00,1,Theft,6000-6099,richmond,ave,-,18F30,210,08/01/09 00:00:00,1,2009,restaurant cafeteria
2,2009-08-01 04:00:00,1,Burglary,3900-3999,broadway,st,-,13D20,210,08/01/09 00:00:00,4,2009,restaurant cafeteria
3,2009-08-01 05:00:00,1,Burglary,8500-8599,bellaire,blvd,-,18F60,210,08/01/09 00:00:00,5,2009,restaurant cafeteria
4,2009-08-01 19:00:00,1,Theft,1100-1199,uptown park,blvd,-,18F20,210,08/01/09 00:00:00,19,2009,restaurant cafeteria


# Load Weather Data

In [51]:
raw = bucket_raw_path('dend-data',f'capstone/inter-data/weather-data/')
raw

's3://dend-data/capstone/inter-data/weather-data/'

# List objects in bucket

In [52]:
wr.s3.list_objects(raw)

['s3://dend-data/capstone/inter-data/weather-data/weather-09-18.csv']

# Create dataframe from s3 files

In [53]:
%%time
df_weather = s3_files_to_df(raw)

CPU times: user 387 ms, sys: 66.2 ms, total: 453 ms
Wall time: 7.41 s


In [54]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87624 entries, 0 to 87623
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            87624 non-null  object 
 1   temp                 87624 non-null  float64
 2   feels_like           87624 non-null  float64
 3   temp_min             87624 non-null  float64
 4   temp_max             87624 non-null  float64
 5   humidity_per         87624 non-null  int64  
 6   wind_speed           87624 non-null  float64
 7   rain_vol_1h_mm       87624 non-null  float64
 8   snow_vol_1h_mm       87624 non-null  float64
 9   clouds_all_per       87624 non-null  int64  
 10  weather_main         87624 non-null  object 
 11  weather_description  87624 non-null  object 
dtypes: float64(7), int64(2), object(3)
memory usage: 8.0+ MB


In [55]:
df_weather.loc[df_weather.duplicated(keep='first'),:]

,date_time,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description


In [56]:
df_weather.loc[df_weather.duplicated(subset=['date_time'],keep=False),:]

,date_time,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description


In [57]:
df_crime.reset_index(drop=True,inplace=True)
df_weather.reset_index(drop=True,inplace=True)

# Weather Data
Total rows: $151,265$

In [58]:
df_weather.head()

,date_time,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2009-01-01 01:00:00,58.622,56.210,55.274,59.702,43,5.1,0.0,0.0,1,Clear,sky is clear
1,2009-01-01 02:00:00,56.768,54.212,53.294,57.902,44,7.2,0.0,0.0,1,Clear,sky is clear
2,2009-01-01 03:00:00,55.166,52.412,52.214,56.102,43,6.2,0.0,0.0,1,Clear,sky is clear
3,2009-01-01 04:00:00,53.330,50.522,50.234,54.302,46,4.6,0.0,0.0,1,Clear,sky is clear
4,2009-01-01 05:00:00,51.188,48.308,50.702,52.034,49,3.6,0.0,0.0,1,Clear,sky is clear


In [59]:
# set to datetime
df_weather.date_time = pd.to_datetime(df_weather.date_time)

In [41]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87624 entries, 0 to 87623
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_time            87624 non-null  datetime64[ns]
 1   temp                 87624 non-null  float64       
 2   feels_like           87624 non-null  float64       
 3   temp_min             87624 non-null  float64       
 4   temp_max             87624 non-null  float64       
 5   humidity_per         87624 non-null  int64         
 6   wind_speed           87624 non-null  float64       
 7   rain_vol_1h_mm       87624 non-null  float64       
 8   snow_vol_1h_mm       87624 non-null  float64       
 9   clouds_all_per       87624 non-null  int64         
 10  weather_main         87624 non-null  object        
 11  weather_description  87624 non-null  object        
dtypes: datetime64[ns](1), float64(7), int64(2), object(2)
memory usage: 8.0+ MB


In [60]:
# set as index
#df_weather = df_weather.set_index('date_time')

In [61]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87624 entries, 0 to 87623
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_time            87624 non-null  datetime64[ns]
 1   temp                 87624 non-null  float64       
 2   feels_like           87624 non-null  float64       
 3   temp_min             87624 non-null  float64       
 4   temp_max             87624 non-null  float64       
 5   humidity_per         87624 non-null  int64         
 6   wind_speed           87624 non-null  float64       
 7   rain_vol_1h_mm       87624 non-null  float64       
 8   snow_vol_1h_mm       87624 non-null  float64       
 9   clouds_all_per       87624 non-null  int64         
 10  weather_main         87624 non-null  object        
 11  weather_description  87624 non-null  object        
dtypes: datetime64[ns](1), float64(7), int64(2), object(2)
memory usage: 8.0+ MB


In [62]:
df_weather.head()

,date_time,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2009-01-01 01:00:00,58.622,56.210,55.274,59.702,43,5.1,0.0,0.0,1,Clear,sky is clear
1,2009-01-01 02:00:00,56.768,54.212,53.294,57.902,44,7.2,0.0,0.0,1,Clear,sky is clear
2,2009-01-01 03:00:00,55.166,52.412,52.214,56.102,43,6.2,0.0,0.0,1,Clear,sky is clear
3,2009-01-01 04:00:00,53.330,50.522,50.234,54.302,46,4.6,0.0,0.0,1,Clear,sky is clear
4,2009-01-01 05:00:00,51.188,48.308,50.702,52.034,49,3.6,0.0,0.0,1,Clear,sky is clear


In [63]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121231 entries, 0 to 1121230
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   date_time            1121231 non-null  datetime64[ns]
 1   offenses             1121231 non-null  int64         
 2   offense_type         1121231 non-null  object        
 3   block_range          1121231 non-null  object        
 4   street_name          1121231 non-null  object        
 5   type                 1121231 non-null  object        
 6   suffix               1121231 non-null  object        
 7   beat                 1121231 non-null  object        
 8   premise              1121231 non-null  object        
 9   date                 1121231 non-null  object        
 10  hour                 1121231 non-null  int64         
 11  year                 1121231 non-null  int64         
 12  premise_description  1121231 non-null  object        
dt

# MERGE

In [64]:
df = pd.merge(df_crime, df_weather,on=['date_time'])
df.info()
# crime: 1121231
# new : 1162993

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121231 entries, 0 to 1121230
Data columns (total 24 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   date_time            1121231 non-null  datetime64[ns]
 1   offenses             1121231 non-null  int64         
 2   offense_type         1121231 non-null  object        
 3   block_range          1121231 non-null  object        
 4   street_name          1121231 non-null  object        
 5   type                 1121231 non-null  object        
 6   suffix               1121231 non-null  object        
 7   beat                 1121231 non-null  object        
 8   premise              1121231 non-null  object        
 9   date                 1121231 non-null  object        
 10  hour                 1121231 non-null  int64         
 11  year                 1121231 non-null  int64         
 12  premise_description  1121231 non-null  object        
 1

In [65]:
df.reset_index(drop=True,inplace=True)

In [66]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2009-08-01,1,Robbery,3000-3099,durham,dr,N,3B30,210,08/01/09 00:00:00,0,2009,restaurant cafeteria,85.784,96.746,82.274,89.114,75,6.2,0.0,0.0,1,Clear,sky is clear
1,2009-08-01,1,Robbery,500-599,seminar,dr,-,6B60,20A,08/01/09 00:00:00,0,2009,apartment,85.784,96.746,82.274,89.114,75,6.2,0.0,0.0,1,Clear,sky is clear
2,2009-08-01,1,Aggravated Assault,600-699,berry,rd,-,3B50,20A,08/01/09 00:00:00,0,2009,apartment,85.784,96.746,82.274,89.114,75,6.2,0.0,0.0,1,Clear,sky is clear
3,2009-08-01,1,Burglary,9700-9799,court glen,dr,-,19G40,20A,08/01/09 00:00:00,0,2009,apartment,85.784,96.746,82.274,89.114,75,6.2,0.0,0.0,1,Clear,sky is clear
4,2009-08-01,1,Auto Theft,5400-5499,newcastle,-,-,1A50,20A,08/01/09 00:00:00,0,2009,apartment,85.784,96.746,82.274,89.114,75,6.2,0.0,0.0,1,Clear,sky is clear


In [67]:
df_crime.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description
0,2009-08-01 00:00:00,1,Robbery,3000-3099,durham,dr,N,3B30,210,08/01/09 00:00:00,0,2009,restaurant cafeteria
1,2009-08-01 01:00:00,1,Theft,6000-6099,richmond,ave,-,18F30,210,08/01/09 00:00:00,1,2009,restaurant cafeteria
2,2009-08-01 04:00:00,1,Burglary,3900-3999,broadway,st,-,13D20,210,08/01/09 00:00:00,4,2009,restaurant cafeteria
3,2009-08-01 05:00:00,1,Burglary,8500-8599,bellaire,blvd,-,18F60,210,08/01/09 00:00:00,5,2009,restaurant cafeteria
4,2009-08-01 19:00:00,1,Theft,1100-1199,uptown park,blvd,-,18F20,210,08/01/09 00:00:00,19,2009,restaurant cafeteria


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121231 entries, 0 to 1121230
Data columns (total 24 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   date_time            1121231 non-null  datetime64[ns]
 1   offenses             1121231 non-null  int64         
 2   offense_type         1121231 non-null  object        
 3   block_range          1121231 non-null  object        
 4   street_name          1121231 non-null  object        
 5   type                 1121231 non-null  object        
 6   suffix               1121231 non-null  object        
 7   beat                 1121231 non-null  object        
 8   premise              1121231 non-null  object        
 9   date                 1121231 non-null  object        
 10  hour                 1121231 non-null  int64         
 11  year                 1121231 non-null  int64         
 12  premise_description  1121231 non-null  object        
 1

In [69]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121231 entries, 0 to 1121230
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   date_time            1121231 non-null  datetime64[ns]
 1   offenses             1121231 non-null  int64         
 2   offense_type         1121231 non-null  object        
 3   block_range          1121231 non-null  object        
 4   street_name          1121231 non-null  object        
 5   type                 1121231 non-null  object        
 6   suffix               1121231 non-null  object        
 7   beat                 1121231 non-null  object        
 8   premise              1121231 non-null  object        
 9   date                 1121231 non-null  object        
 10  hour                 1121231 non-null  int64         
 11  year                 1121231 non-null  int64         
 12  premise_description  1121231 non-null  object        
dt